In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/Active_v0.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])
    
    
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(normalize_data=False)

indices = list(range(len(x_train)))
random.seed(101)
random.shuffle(indices)
labeled_set = indices[:config["RUNS"]["ADDENDUM"] ]
unlabeled_set = indices[config["RUNS"]["ADDENDUM"] :]

# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

print(config)

from inference_agent_cifar_keras import Active_Learning_inference



2020-12-15 15:49:19,782	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0}, 'DATASET': {'height_shift_range': 4, 'horizontal_flip': True, 'width_shift_range': 4}, 'NETWORK': {'CLASSES': ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], 'INPUT_SIZE': 32, 'MARGIN': 1.0, 'embedding_size': 128}, 'PROJECT': {'Backbone': 'resnet18', 'dataset_name': 'CIFAR', 'group': 'Active_Learning_keras_v0', 'group_dir': '/mnt/Ressources/Andres/Temp_active/runs/Active_Learning_keras_v0', 'project': 'Active_Learning_CIFAR', 'project_dir': '/mnt/Ressources/Andres/Temp_active/runs', 'source': 'CIFAR'}, 'RUNS': {'ADDENDUM': 1000, 'CYCLES': 10, 'SUBSET': -1, 'TRIALS': 1, 'test_each': 5}, 'TEST': {'batch_size': 128}, 'TRAIN': {'Data_augementation': True, 'EPOCH_SLIT': 80, 'EPOCH_WARMUP': 2, 'EPOCH_WHOLE': 120, 'MILESTONES': [160], 'batch_size': 128, 'gamma': 0.1, 'lr': 0.001, 'start_epoch': 0, 'transfer_weight_path': False, 'w_c_loss': 1.0, 'w_l_loss': 1.0, 'wdecay': 0.9995}}


In [2]:
num_run = 0
resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run),'checkpoints','checkpoint.200.hdf5')
network = Active_Learning_inference.remote(config , unlabeled_set, num_run, resume_model_path)

In [3]:
network.inference.remote()